Quests 2 - Modelos 1 - Disípulos de Hemera
===================

## Introdução



Para a resolução da quest, utilizamos como base o código utilizado em sala de aula, alterando o tipo de modelo, de regressor linear para classificador linear. Além disso, mudamos a métrica da distância para Manhattan e por fim testamos a eficácia do nosso modelo.

### Distância de Manhattan
"A distância de Manhattan entre dois pontos em um espaço euclidiano com sistema de coordenadas cartesianas fixado é a soma dos comprimentos das projeções do segmento de reta que liga os pontos sobre os eixos coordenados. Por exemplo, no plano, a taxi-distancia entre o ponto P1 com coordenadas (x1, y1) e o ponto P2 em (x2, y2) é |x1 - x2| + |y1 - y2|." **[1]**

$$ Distância  de Manhattan = |x_1-x_2| + |y_1 - y_2| + |z_1 - z_2| + ... + |v_1 - v_2| $$

## Desenvolvimento



#### Importando as bibliotecas necessárias:

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

#### Definindo os dados:
Como pedido pela quest, usaremos o dataset de pinguins do seaborn. **[2]**

In [2]:
dados = sns.load_dataset('penguins')
dados = dados.dropna()

In [3]:
dados

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


#### Atribuindo os Targets e Features:
Como pedido na quest, atribuímos o atributo target como as espécies.

In [4]:
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]
TARGET = ["species"]

X = dados.reindex(FEATURES, axis=1).values
y = dados.reindex(TARGET, axis=1).values.ravel()

#### Utilizando o algoritmo classificador k-NN:
Utilizamos o n de vizinhos como 3 e a métrica como manhattan como pedido na quest. Para isso, alteramos a função calcula distância, para retornar a distância de Manhattan (como temos duas coordenadas). Após isso, por nosso target ser categórico, associamos o y_previsto não ao valor médio dos vizinhos, mas sim à moda. **[3]**

In [5]:
import statistics as st
 
def calcula_distancia(coordenada_1, coordenada_2):
    distancia = 0
    for c1, c2 in zip(coordenada_1, coordenada_2): # zip "liga" os dois valores
        distancia += abs(c1-c2)
    return distancia
 
def treinar_knn(modelo, X, y,num_vizinhos):
    modelo["features"] = X
    modelo["target"] = y
    modelo["num_vizinhos"] = num_vizinhos
    
def previsao_knn(modelo, X):
    distancias = []
    for pinguim in modelo["features"]:
        distancia_encontrada = calcula_distancia(pinguim, X)
        distancias.append(distancia_encontrada)
    indices = np.argsort(distancias)[:modelo["num_vizinhos"]] # busca o índice dos pinguins mais próximos e "corta" a lista para a quantidade de vizinhos selecionada
    targets = modelo["target"][indices]
    y_previsto = st.mode(targets)
    
    return y_previsto

Agora vamos treinar o nosso modelo e testá-lo para o pinguim hipotético:

In [6]:
modelo = {}
 
treinar_knn(modelo, X, y, num_vizinhos=3)

In [7]:
x_novo = [36, 21, 182]
 
y_previsto = previsao_knn(modelo, x_novo)
 
print(y_previsto)

Adelie


#### Alternativamente poderíamos ter feito: 
Poderíamos utilizar a função do sci-kit learn para criar um modelo k-NN Classificador. **[4]**
Após isso vamos testar a eficácia do modelo, para isso usamos a estratégia de split de dados.**[5]** 

Escolhemos o tamanho de teste em 20% e a seed aleatória atribuímos 1. Demos o split nos índices e iteramos a partir da função loc.

In [8]:
from sklearn.model_selection import train_test_split

indices_treino, indices_teste = train_test_split(
    dados.index, test_size = 0.2, random_state = 1
)

dados_treino = dados.loc[indices_treino]
dados_teste = dados.loc[indices_teste]

In [9]:
dados_treino

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
306,Gentoo,Biscoe,43.4,14.4,218.0,4600.0,Female
168,Chinstrap,Dream,50.3,20.0,197.0,3300.0,Male
181,Chinstrap,Dream,52.8,20.0,205.0,4550.0,Male
167,Chinstrap,Dream,50.5,19.6,201.0,4050.0,Male
19,Adelie,Torgersen,46.0,21.5,194.0,4200.0,Male
...,...,...,...,...,...,...,...
209,Chinstrap,Dream,49.3,19.9,203.0,4050.0,Male
262,Gentoo,Biscoe,45.3,13.7,210.0,4300.0,Female
78,Adelie,Torgersen,36.2,16.1,187.0,3550.0,Female
241,Gentoo,Biscoe,45.1,14.5,215.0,5000.0,Female


In [10]:
dados_teste

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
65,Adelie,Biscoe,41.6,18.0,192.0,3950.0,Male
276,Gentoo,Biscoe,43.8,13.9,208.0,4300.0,Female
186,Chinstrap,Dream,49.7,18.6,195.0,3600.0,Male
198,Chinstrap,Dream,50.1,17.9,190.0,3400.0,Female
293,Gentoo,Biscoe,46.5,14.8,217.0,5200.0,Female
...,...,...,...,...,...,...,...
255,Gentoo,Biscoe,48.4,16.3,220.0,5400.0,Male
138,Adelie,Dream,37.0,16.5,185.0,3400.0,Female
86,Adelie,Dream,36.3,19.5,190.0,3800.0,Male
79,Adelie,Torgersen,42.1,19.1,195.0,4000.0,Male


In [11]:
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]
TARGET = ["species"]

X_treino = dados_treino.reindex(FEATURES, axis=1).values
y_treino = dados_treino.reindex(TARGET, axis=1).values.ravel()

X_teste = dados_teste.reindex(FEATURES, axis=1).values
y_teste = dados_teste.reindex(TARGET, axis=1).values.ravel()

In [12]:
modelo_knn = KNeighborsClassifier(n_neighbors=3, metric = 'manhattan')
modelo_knn.fit(X_treino, y_treino)

y_previsto = modelo_knn.predict(X_teste)

Ou:

In [13]:
modelo_knn = KNeighborsClassifier(n_neighbors=3, p = 1)
modelo_knn.fit(X_treino, y_treino)

y_previsto = modelo_knn.predict(X_teste)

Utilizamos a relação entre os acertos do algoritmos com o número de previsões, do modelo dado pela função do sci-kit para testarmos sua eficiência.

In [14]:
def testa_acertos(y_teste, y_previsto):
    acertos = 0
    for i in range (0,len(y_teste)):
        if y_teste[i] == y_previsto[i]:
            acertos += 1
    print (f"A taxa de acerto foi de: {round(acertos/len(y_teste)*100,2)}%")

testa_acertos(y_teste,y_previsto)

A taxa de acerto foi de: 98.51%


## Conclusão



Ao chegarmos na Taverna do Sol, fomos abordados pelo Cavaleiro Luminoso, que nos deu uma quest na qual melhoramos nossos atributos de destreza ao nos auxiliar na predição de movimentos de adversários (utilizar modelos de predição para targets categóricos) e cálculo de distâncias que nos ajudará na pontaria (cálculo de distânc. Infelizmente, ao concluírmos nossa quest não pudemos nos encontrar com o Cavaleiro Luminoso, pois o dia estava nublado e a taverna estava fechada. Porém, na semana seguinte conseguimos nos encontrar com o Cavaleiro Luminoso, e após bebermos para comemorar o sucesso da quest, fomos expulsos da taverna, pois esquecemos de levantar os braços lentamente (desta forma:
\ o/) em direção ao sol.

## Referências



[1] Referência sobre a distância de Manhattan ou geometria do táxi: https://pt.wikipedia.org/wiki/Geometria_do_t%C3%A1xi

[2] Github com datasets didáticos: https://github.com/mwaskom/seaborn-data

[3] Daniel Cassar, material de aula: 'ATP-203 2.1 - Aprendizado de máquina, k-NN e métricas'

[4] Documentação Scikit Learn sobre KNeighbors Classifier: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

[5] Daniel Cassar, material de aula: 'ATP-203 4.0 - Split de dados de treino e teste'